In [3]:
 from azureml.core import Workspace, Dataset,Experiment
from sklearn.metrics import accuracy_score


 subscription_id = 'a6632a9d-4bd9-4aa0-99ce-0d47bc549995'
resource_group = 'mlops-august'
workspace_name = 'intellipat-mlops'

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [6]:
experiment = Experiment(workspace=workspace,name ="iris")
dataset = Dataset.get_by_name(workspace=workspace,name="Iris")


df=dataset.to_pandas_dataframe()
df.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [23]:
X = df[['SepalLengthCm','SepalWidthCm',	'PetalLengthCm','PetalWidthCm']]
Y = df[['Species']]
run = experiment.start_logging()

import joblib
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,shuffle=True)

from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(max_depth=4,criterion="gini")

clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

run.log("accuracy",accuracy*100)

model_name = "iris_decision_tree.pkl"
filepath = "outputs/"+model_name

joblib.dump(value = clf,filename=filepath)
run.upload_file(name=model_name,path_or_stream=filepath)
run.complete()



0.9333333333333333


In [26]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

In [27]:
model= Model.register(
    workspace=workspace,
    model_name="iris",
    model_path=filepath,
    model_framework= Model.Framework.SCIKITLEARN,
    model_framework_version=sklearn.__version__,
    description="This is a simple desiciontree model",
    resource_configuration=ResourceConfiguration(cpu=1,memory_in_gb=0.5)
)

print("Name:",model.name)
print("Version:",model.version)

NameError: name 'sklearn' is not defined